## Mounting the G-drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Loading the Melanoma ISIC 2019 Dataset from Kaggle

In [ ]:
!pip install --upgrade -q kaggle
!mkdir /root/.kaggle
import json
token = {
    "username": "ifrahandleeb",
    "key": "71824991b0fadac07859c3f3fa4314f4"
}

with open('/root/.kaggle/kaggle.json', 'w') as config_file:
    json.dump(token, config_file)
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d wanderdust/skin-lesion-analysis-toward-melanoma-detection

100% 11.3G/11.3G [02:05<00:00, 69.3MB/s]
100% 11.3G/11.3G [02:05<00:00, 96.9MB/s]


In [ ]:
import zipfile
zipref=zipfile.ZipFile("/content/skin-lesion-analysis-toward-melanoma-detection.zip",'r')
zipref.extractall()
zipref.close()

In [ ]:
!rm -rf /content/skin-lesion-analysis-toward-melanoma-detection.zip

##Importing the 'imgaug' library for Data Augmentation

In [ ]:
from imgaug import augmenters as iaa

## [DOCUMENTATION: Read it before the next code cell](https://imgaug.readthedocs.io/en/latest/source/overview/geometric.html#affine)


In [ ]:
import os
from glob import glob
from datetime import datetime
from shutil import copyfile

import imgaug as ia
import imgaug.augmenters as iaa
from keras.preprocessing.image import save_img
from skimage.io import imread


# Input and output directories for image augmentation

INPUT = '/content/skin-lesions/train/seborrheic_keratosis'   # Run for indivdual classes
OUTPUT = '/content/keratosis_aug'
WHITE_LIST_FORMAT = ('png', 'jpg', 'jpeg', 'bmp', 'ppm', 'JPG')
ITERATIONS = 9                                               # Number of times to augment each image

def check_dir_or_create(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.

sometimes = lambda aug: iaa.Sometimes(0.5, aug)

# Define a set of augmentations to be applied with specified probabilities

augmenters = [
     iaa.Fliplr(0.5), #horizontal flip

        iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))), # Strengthen or weaken the contrast in each image.
        iaa.ContrastNormalization((0.75, 1.5)),               # Adjust contrast
        iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),   # Sharpen images
        iaa.AdditiveGaussianNoise(
            loc=0, scale=(0.0, 0.05 * 255), per_channel=0.5)  # Make some images brighter and some darker.
                                                              # In 20% of all cases, we sample the multiplier once per channel,
                                                              # which can end up changing the color of the images.



]

seq = iaa.Sequential(augmenters, random_order=True)           # Create a sequence of augmentations to be applied in random order

files = [y for x in os.walk(INPUT)
         for y in glob(os.path.join(x[0], '*')) if os.path.isfile(y)]
files = [f for f in files if f.endswith(WHITE_LIST_FORMAT)]
classes = [os.path.basename(os.path.dirname(x)) for x in files]
classes_set = set(classes)

# Create directories in the output for each class

for _class in classes_set:
    _dir = os.path.join(OUTPUT, _class)
    check_dir_or_create(_dir)

# Split files into batches for processing

batches = []
BATCH_SIZE = 50
batches_count = len(files) // BATCH_SIZE + 1
for i in range(batches_count):
    batches.append(files[BATCH_SIZE * i:BATCH_SIZE * (i + 1)])

# Augment images and save them to the output directory

images = []
for i in range(ITERATIONS):
    print(i, datetime.time(datetime.now()))
    for batch in batches:
        images = []
        for file in batch:
            img = imread(file)
            images.append(img)
        images_aug = seq.augment_images(images)
        for file, image_aug in zip(batch, images_aug):
            root, ext = os.path.splitext(file)
            new_filename = root + '_{}'.format(i) + ext
            new_path = new_filename.replace(INPUT, OUTPUT, 1)

            save_img(new_path, image_aug)

# Copy original images to the output directory

for file in files:
    dst = file.replace(INPUT, OUTPUT)
    copyfile(file, dst)

## Zipping the Output file

In [ ]:
!zip -r /content/keratosis_aug.zip /content/keratosis_aug

## Saving the Augmented Data on drive

In [ ]:
!cp -r /content/keratosis_aug.zip /content/drive/MyDrive/Skin_lesion_Aug_data/Keratosis

**NOTE**


*   The code should be run for each class separately.


